# Artigo 7 - Filtros Colaborativos aplicado ao dataset *Book-Crossing: User review ratings*

## Objetivo

O objetivo deste presente trabalho é criar um modelo de recomendação de livros usando o dataset *Book-Crossing: User review ratings* aplicando filtros colaborativos aprendido na lição 7 do *fast.ai*.

## Autor

- Levi de Oliveira Queiroz 170108341
- GitHub: LeviQ27
- Kaggle User: lqueiroz27
- HuggingFace User: L27Queiroz

## Referência

- Notebook Kaggle Utilizado como referência para produzir o presente artigo: https://www.kaggle.com/code/jhoward/collaborative-filtering-deep-dive/notebook#Deep-Learning-for-Collaborative-Filtering

- Lição 07 *Collaborative filtering* do *fastai*: https://course.fast.ai/Lessons/lesson7.html

- O Dataset *Book-Crossing: User review ratings* utilizado: https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset

## Inferência

Aplicativo HuggingFace: https://huggingface.co/spaces/L27Queiroz/BookRecomendations

## Desenvolvimento

Comecei importando os módulos necessários para lidar com questões de filtragem colaborativa e com dados tabulares, também coloquei, assim como o Jeremy, uma semente 42 para geração de números aleatórios:

In [1]:
from fastai.collab import *
from fastai.tabular.all import *
set_seed(42)

Feito a importação, comecei a verificar o dado escolhido para este presente trabalho. No local do dado é dito que contém 278,858 usuários (anonimizado mas com informações demográficas) provendo 1,149,780 classificações (explicita ou implicita) de 271,379 livros. Assim, fiz a leitura do arquivo csv *BX-Book-Ratings.csv*:

In [2]:
path = Path('../input/bookcrossing-dataset')
ratings = pd.read_csv(path/'Book reviews/Book reviews/BX-Book-Ratings.csv', encoding='latin-1', delimiter=';', header=None, usecols=(0,1,2), names=('user_id','isbn','rating'), low_memory=False)
ratings = ratings.drop(0)
ratings = ratings.reset_index(drop=True)
ratings.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


Uma observação sobre o que foi feito no código: 
Quando é feito a leituros do arquivo csv *BX-Book-Ratings.csv* pelo `pd.read_csv()` apresenta um erro de leitura *utf-8*, para resolver apliquei como parâmetro `encoding='latin-1'`, que simplismente altera a linguagem de codificação de *utf-8* para *latin-1*. Logo após rodar novamente tive o erro de divisão de colunas, que por padrão, o pandas entende que `,` é um separador de colunas, por isso passei como parâmetro `delimiter=';'` que simplesmente altera o padrão de entendimento de separador do pandas de `,` para `;`. Em seguida tirei os título das colunas usando o parâmetro `header=None`, porém percebi que ao fazer isso a linha *0* era colocada com os títulos das colunas, para resolver isso apliquei a função de remoção da linha *0* usando `drop()` e resetei o index das linhas usando a função `reset_index(drop=True)`, de modo que o index das linhas é resetado e os títulos foram eliminados. Sem os título e seguindo as instruções do Jeremy, separei as colunas *0*, *1* e *2* com `usecols` e inseri os nomes das colunas usando `names`. A utilização do parâmetro `low_memory=False` foi feito por uma sugestão no Kaggle ao rodar o código sem esse parâmetro, pois as colunas *0* e *2* tem tipos misturados, e ao rodar usando esse parâmetro não retornou avisos.

Em seguida, foi feito as etapas para criar o DataLoaders, começando por:

Fiz a leitura do arquivo *BX_Books.csv* para obter os nomes e números *ISBN* de cada um deles,

In [3]:
books = pd.read_csv(path/'Book reviews/Book reviews/BX_Books.csv', encoding='latin-1', delimiter=';', header=None, low_memory=False, usecols=(0,1), names=('isbn','title'))
books = books.drop(0)
books = books.reset_index(drop=True)
books.head()

,isbn,title
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It
4,0393045218,The Mummies of Urumchi


Assim como anteriormente, tratei os dados contidos em `BX_Books.csv` com tratei para criar a variável `ratings`. 

Como instruido na aula do Jeremy, juntei a tabela `ratings` com a tabela `books`: 

In [4]:
ratings = ratings.merge(books)
ratings['rating'] = ratings['rating'].astype('int')
ratings.head()

,user_id,isbn,rating,title
0,276725,034545104X,0,Flesh Tones: A Novel
1,2313,034545104X,5,Flesh Tones: A Novel
2,6543,034545104X,0,Flesh Tones: A Novel
3,8680,034545104X,5,Flesh Tones: A Novel
4,10314,034545104X,9,Flesh Tones: A Novel


Agora, criei um objeto `DataLoaders` dessa tabela. Por padrão,as colunas *user_id*, *isbn* e *rating* serão utilizadas para produzir o objeto. Assim, mudei o valor de `item_name` no nosso caso para usar os títulos em vez dos IDs:

In [5]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=824288)
dls.show_batch()

,user_id,title,rating
0,143253,The Cunning Man: A Novel,0
1,115490,Briar Rose,0
2,76352,Lake News,0
3,17190,"Left Behind Graphic Novel (Book 1, Vol.3 )",7
4,275970,The New Russians: Updated to Include the Failed Coup,0
5,3363,Step-Ball-Change,0
6,242824,"Charmed Circle (American Romance, No 301)",0
7,269136,Penguin Readers Level 2: of Mice and Men (Penguin Readers),0
8,258907,"The Metamorphosis, In the Penal Colony, and Other Stories",0
9,169357,Writ of Execution,9


Descrevendo um pouco sobre o código, foi cirado o objeto `dls` que contém os dados necessários para treinar e validar o modelo de filtragem colaborativa. `CollabDataLoaders` é uma classe fornecida pelo fastai qua ajuda na criação dos `DataLoaders` que são responsáveis por fornecer os lotes de dados para o treinamento do modelo. O método estático `from_df` permite criar um objeto a partir de `ratings`. O parâmetro `item_name='title'` indica a coluna de `ratings` que contém os nomes dos itens, é necessário para que o modelo saiba como identificar cada item. O parâmetro `bs=824288` define o tamanho do lote usado durante o treinamento do modelo. O comando `dls.show_batch()` exibe uma amostra dos dados carregados nos `DataLoaders`.

Para representar filtragem colaborativa em PyTorch não posso apenas usar a representação apresentada diretamente, especialmente quando quero encaixar o framework deep learning que vou usar. Representei as tabelas de fatores latentes de livros e usuários como matrizes simples:

In [6]:
n_users = len(dls.classes['user_id'])
n_books = len(dls.classes['title'])
n_factors = 50

As variáveis `n_users` e `n_books` recebem o número de usuários e de livros no conjunto de dados. Isso pois foi utilizado a função `len()` para calcular o comprimento das colunas, selecionadas através de `dls.classes[coluna]`. Coloquei como *50* fatores latentes.

Agora como parte da lição 7, o Jeremy passa pela criação do produto escalar por etapas. No caso, em referência à lição 7 fiz uma função de produto escalar:

In [7]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 10)):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.book_factors = Embedding(n_books, n_factors)
        self.book_bias = Embedding(n_books, 1)
        self.y_range = y_range
    
    def forward(self, x):
        users = self.user_factors(x[:,0])
        books = self.book_factors(x[:,1])
        res = (users * books).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:,0]) + self.book_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)

No código acima tenho a importação da classe `Module` do módulo `torch.nn` do PyTorch que é usada para definir modelos de aprendizagem de máquina, dela é fronecido funcionalidades e métodos comuns para criar e gerenciar modelos. Assim, criei a classe `DotProductBias`, com `__init__` sendo construtor da classe que inicia os atributos da classe e define as camadas e parâmetros necessários para o modelo, `n_users` é o número de usuários e `n_books` de livros do conjunto dados, `n_factors` ~e o número de fatores latentes a serem usados, no caso definir como 5, `y_range` é um intervalo de valores esperados para as previsôes do modelo. Para as partes usando o `Embedding` estou fazendo uma incorporação para representar os usuários e os livros, no caso de `*_factors` estou querendo usar a camada de incorporação para representar os usuários e livros, no caso de `*_bias`estou querendo representar o viés dos usuários e livros. Seguindo para `foward()` este metodo define a passagem direta do modelo, sendo *x* um tensor de entrada que contém os índices dos usuários e filme, `users` e `books` usam as camadas de incorporação para obter os fatores latentes correspodentes a cada um dele presente em `x[:,0]` e `x[:,1]`. Assim faço a some do produto escalar dos fatores latentes de `users` e `books` em `(users * books).sum(dim=1, keepdim=True)` ao longo da dimensão 1 `dim=1` e matenho as dimensões da matriz em `keepdim=True`. O resultado dessa soma é uma matriz de previsões parciais para cada par usuário-livro. Assim sigo para adicionar os viéses dos usuário e dos filme às previsões parcias, auxiliando as previsões de acordo com os padrões específicos de cada usuário e livro em `res += self.user_bias(x[:,0]) + self.book_bias(x[:,1])`. Com isso feito, dou um retorno da função `sigmoid_range` em que é aplicado uma função de ativação sigmoidal às previsões resultantes e aplica um redimensionamento linear para garantir que as previsões estejam dentro do intervalo definidao por `y_range`.

Agora treino o modelo aplicando o conceito de Decaimento por peso, que consiste em adicionar para a função de loss a soma de todos os pasos ao quadrado, para encorajar o pesos a serem o mais pequenos possíveis:

In [8]:
model = DotProductBias(n_users, n_books, n_factors)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(20, 10e-2, wd=0.1)

epoch,train_loss,valid_loss,time
0,19.532913,19.462479,00:04
1,19.481537,19.226902,00:04
2,19.361034,18.683155,00:04
3,19.117989,17.847898,00:04
4,18.735947,16.765671,00:04
5,18.199915,15.483126,00:04
6,17.493793,14.224391,00:04
7,16.642176,13.353561,00:04
8,15.720423,12.937882,00:04
9,14.798668,12.795275,00:04


No código acima, criei uma instância do modelo `DotPRoductBias` com base no número de usuários, número de livros e 50 fatores latentes. Em seguida, ele cria um objeto `Learner` com base nos dados de treinamento, no modelo e a função de perda `MSELossFlat`. Depois disso apliquei o método `fit_one_cycle` para treinar o modelo por 5 épocas, com uma taxa de aprendizagem inicial de ... e uma peso de decaimento (weight decay) de 0.1.

Agora para finalizar a parte do desenvolvimento do artigo e da produção do modelo, criei um modelo deep learning para a filtragem colaborativa. Para isso, comecei colocando na variável `embs` o retorno da recomendação da função do *fastai* `get_emb_sz` dá de tamanho de incorporação de matrizes para o dado que estou usando,

In [9]:
embs = get_emb_sz(dls)
embs

[(92108, 600), (241091, 600)]

Agora implemento essa classe:

In [10]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,11), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range
        
    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

A classe `CollabNN` implementa um modelo de filtragem colaborativa com redes neurais. Esse modelo é usado para fazer previsões ou recomendações com base em dados de usuários e itens, analisando como ele funciona tenho:

No método `__init__`, são definidos os atributos da classe. `user_sz` e `item_sz` são tuplas que contêm o número de usuários e itens no conjunto de dados, respectivamente, bem como a dimensão dos fatores latentes para usuários e itens. O intervalo `y_range` especifica os valores esperados das previsões do modelo, e `n_act` determina o número de neurônios na camada oculta da rede neural.

Em seguida, são criadas as camadas de incorporação (embedding) para representar os usuários e itens. A camada de incorporação para os usuários tem dimensão de acordo com `user_sz`, e a camada de incorporação para os itens tem dimensão de acordo com `item_sz`.

A rede neural é definida usando a classe `nn.Sequential`, que permite empilhar camadas sequencialmente. Ela consiste em uma camada linear de entrada, uma função de ativação `ReLU` e uma camada linear de saída. A camada linear de entrada recebe as representações concatenadas dos usuários e itens como entrada, enquanto a camada linear de saída produz uma única saída, que é a previsão final do modelo.

No método `forward`, o tensor de entrada *x* contém os índices dos usuários e itens. A partir desses índices, são obtidas as representações de incorporação dos usuários e itens correspondentes. Essas representações são então concatenadas ao longo da dimensão 1 para formar um único tensor de entrada para a rede neural.

Esse tensor é passado pelas camadas da rede neural definidas anteriormente, e a saída resultante é aplicada a uma função de ativação sigmoidal. Essa função sigmoidal garante que as previsões estejam dentro do intervalo especificado por `y_range`. Por fim, as previsões redimensionadas são retornadas como saída do método forward.

E assim, utilizo para criar o modelo e fazer o treino dele:

In [11]:
model = CollabNN(*embs)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(15, 10e-2, wd=0.01)

epoch,train_loss,valid_loss,time
0,20.607233,20.228888,00:33
1,20.407421,15.414684,00:29
2,18.643677,16.505472,00:29
3,17.803856,19.885031,00:30
4,17.729347,16.281061,00:29
5,17.269590,16.207882,01:16
6,16.665724,16.746372,00:54
7,16.454834,15.700248,00:51
8,16.031324,15.990754,00:48
9,15.666648,15.449700,00:44


Essas linhas de código têm o objetivo de criar e treinar um modelo de filtragem colaborativa. Entendendo o que cada parte faz:

Na linha `model = CollabNN(*embs)`, um objeto chamado model é criado. Ele representa o modelo de filtragem colaborativa e utiliza as representações de incorporação dos usuários e itens, que estão armazenadas na variável embs. O desempacotamento `(*embs)` permite passar as representações corretas para o construtor da classe *CollabNN*, inicializando o modelo adequadamente.

Na linha `learn = Learner(dls, model, loss_func=MSELossFlat())`, um objeto chamado learn é criado. Ele representa o objeto de aprendizado (learner) responsável por treinar o modelo. O objeto dls contém o conjunto de dados de treinamento e validação. O modelo criado anteriormente (model) é passado como argumento, juntamente com a função de perda *MSELossFlat()*, que mede o erro quadrático médio entre as previsões do modelo e os rótulos reais.

Na linha `learn.fit_one_cycle(15, 10e-2, wd=0.01)`, o método `fit_one_cycle()` é chamado para treinar o modelo. Ele recebe três argumentos: o número de épocas de treinamento (15), a taxa de aprendizado (10e-2) e a força da regularização (0.01). Durante o treinamento, o modelo será ajustado aos dados em várias iterações (épocas) para fazer previsões mais precisas.

Feito isso, passei para a exportação do modelo para utilizá-lo no aplicativo do Hugging Face:

In [16]:
learn.export('model.pkl')

## Conclusão

Achei interessante construir um modelo de recomendação utilizando filtros colaborativos e também presenciei bastantes dificuldades com o dataset, tive problemas de linguagem *encoding*, a coluna de ratings estava vindo como objetos e o *DataLoader* só trabalhos com valores númericos, na hora de usar o modelo em produto escalar teve um momento em que fique 1 hora esperando o fazer o primeiro ciclo de treinamento, no HuggingFace App na hora de pedir para fazer a sugestão está dando error. Então finalizando, foi trabalhoso e divertido estudar sobre filtros colaborativos e gradientes descedentes.
